In [11]:
import os



In [3]:
!git clone -b jeremy --single-branch https://github.com/Jalalbaim/Direct-Preference-Optimization.git


Cloning into 'Direct-Preference-Optimization'...
remote: Enumerating objects: 169, done.
remote: Counting objects: 100% (151/151), done.
remote: Compressing objects: 100% (98/98), done.
remote: Total 169 (delta 82), reused 109 (delta 51), pack-reused 18 (from 1)
Receiving objects: 100% (169/169), 1.27 MiB | 18.33 MiB/s, done.
Resolving deltas: 100% (82/82), done.


In [2]:
rm -rf Direct-Preference-Optimization

In [10]:
ls

In [4]:
# Changer de répertoire vers le projet cloné
%cd Direct-Preference-Optimization/


import os
print(f" Répertoire actuel: {os.getcwd()}")
print(f" Contenu: {os.listdir('.')[:10]}")

/content/Direct-Preference-Optimization
 Répertoire actuel: /content/Direct-Preference-Optimization
 Contenu: ['train_ppo.ipynb', 'configs', 'src', '.git', 'requirements.txt', '.gitignore', 'README.md', 'scripts', 'data']


In [5]:
%%capture
# Installation silencieuse des packages nécessaires
!pip install torch transformers datasets accelerate bitsandbytes sentencepiece protobuf tqdm pyyaml scikit-learn

### Étape 2: Installation des dépendances

In [12]:

from google.colab import drive
drive.mount('/content/drive')
    
    

ValueError: mount failed

## 1. Setup et Imports

In [6]:
import os
import sys
import torch
from torch.utils.data import DataLoader

# Ajouter le répertoire racine au path (déjà dans le bon répertoire après %cd)
ROOT = os.path.abspath('.')
if ROOT not in sys.path:
    sys.path.append(ROOT)

print(f" Working directory: {os.getcwd()}")

from src.core.models import load_models
from src.core.data import PromptDataset, prompt_collate_fn
from src.ppo.ppo_trainer import PPOTrainer
from src.core.utils import load_yaml_config

print("Imports réussis")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

 Working directory: /content/Direct-Preference-Optimization
Imports réussis
PyTorch version: 2.9.0+cu126
CUDA available: True


In [ ]:
import importlib.util
import os

# Vérifier/install wandb
if importlib.util.find_spec("wandb") is None:
    print("Installing wandb...")
    %pip install -q wandb

try:
    import wandb
    WANDB_AVAILABLE = True
except Exception as e:
    WANDB_AVAILABLE = False
    print(f"wandb import failed: {e}")

# Login si la clé est présente dans l'env
if WANDB_AVAILABLE:
    if os.environ.get("WANDB_API_KEY"):
        wandb.login(relogin=True)
        print("wandb logged in (env key)")
    else:
        print("⚠️  WANDB_API_KEY non défini. Ajoute la clé dans l'env ou exécute wandb.login().")


In [7]:
# Vérifier si les données de prompts existent
prompts_path = "data/processed/sentiment/prompts.jsonl"

if not os.path.exists(prompts_path):
    print(" Fichier prompts.jsonl non trouvé!")
    print("Exécution de prepare_prompts.py...")
    !python scripts/prepare_prompts.py
    print("Prompts préparés")
else:
    # Compter le nombre de prompts
    with open(prompts_path, 'r') as f:
        num_prompts = sum(1 for line in f if line.strip())
    print(f"{num_prompts} prompts trouvés dans {prompts_path}")

2497 prompts trouvés dans data/processed/sentiment/prompts.jsonl


## 4. Configuration des Hyperparamètres

Vous pouvez modifier ces paramètres selon vos besoins:

In [8]:
# Charger la config par défaut
config_path = "configs/ppo_sentiment.yaml"
config = load_yaml_config(config_path)

# Afficher les paramètres principaux
print(" Configuration PPO:")
print(f"  Model: {config['model']['name']}")
print(f"  Batch size: {config['training']['batch_size']}")
print(f"  Learning rate: {config['training']['learning_rate']}")
print(f"  Epochs: {config['training']['num_epochs']}")
print("\n Paramètres PPO:")
print(f"  Clip epsilon: {config['ppo']['clip_epsilon']}")
print(f"  Value coef: {config['ppo']['value_coef']}")
print(f"  Entropy coef: {config['ppo']['entropy_coef']}")
print(f"  Target KL: {config['ppo']['target_kl']}")
print(f"  PPO epochs: {config['ppo']['num_ppo_epochs']}")
print("\n Génération:")
print(f"  Max length: {config['generation']['max_length']}")
print(f"  Temperature: {config['generation']['temperature']}")

 Configuration PPO:
  Model: TinyLlama/TinyLlama-1.1B-Chat-v1.0
  Batch size: 2
  Learning rate: 1e-5
  Epochs: 1

 Paramètres PPO:
  Clip epsilon: 0.2
  Value coef: 0.5
  Entropy coef: 0.01
  Target KL: 0.01
  PPO epochs: 4

 Génération:
  Max length: 128
  Temperature: 0.7


### Modifier les paramètres (optionnel)

Décommentez et modifiez si vous voulez changer certains paramètres:

In [9]:
# Exemple de modifications
# config['training']['batch_size'] = 1  # Réduire si problème de mémoire
# config['training']['num_epochs'] = 2  # Plus d'epochs
# config['ppo']['num_ppo_epochs'] = 2   # Moins d'epochs PPO par batch
# config['generation']['max_length'] = 64  # Réponses plus courtes

print("⚙️  Configuration personnalisée (si modifiée)")

⚙️  Configuration personnalisée (si modifiée)


## 5. Chargement des Modèles

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [11]:
model_name = config["model"]["name"]
dtype = config["model"]["dtype"]

# Charger les modèles (policy et référence)
mb = load_models(model_name, dtype=dtype, device=device)
tokenizer = mb.tokenizer

print(f" Modèles chargés: {model_name}")
print(f"   Policy model: {mb.policy_model.num_parameters():,} paramètres")
print(f"   Device: {mb.device}")

# 🔍 Diagnostic: Vérifier que tout est sur GPU
print(f"\n Diagnostic device:")
print(f"   Policy: {next(mb.policy_model.parameters()).device}")
print(f"   Ref: {next(mb.ref_model.parameters()).device}")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(
`torch_dtype` is deprecated! Use `dtype` instead!


 Modèles chargés: TinyLlama/TinyLlama-1.1B-Chat-v1.0
   Policy model: 1,100,048,384 paramètres
   Device: cuda

 Diagnostic device:
   Policy: cuda:0
   Ref: cuda:0


## 6. Préparation du DataLoader

In [12]:
# Charger le dataset de prompts
prompt_dataset = PromptDataset(config["data"]["prompt_path"])
max_prompt_length = config["data"]["max_prompt_length"]

print(f"Dataset chargé: {len(prompt_dataset)} prompts")

# Fonction de collate
def collate(batch):
    return prompt_collate_fn(
        batch,
        tokenizer=tokenizer,
        max_prompt_length=max_prompt_length,
    )

# DataLoader
prompt_loader = DataLoader(
    prompt_dataset,
    batch_size=config["training"]["batch_size"],
    shuffle=True,
    collate_fn=collate,
)

print(f" DataLoader créé: {len(prompt_loader)} batches")

Dataset chargé: 2497 prompts
 DataLoader créé: 1249 batches


## 7. Initialisation du PPO Trainer

In [13]:
print("⚙️ Initialisation du PPO Trainer...")

# Créer le trainer
trainer = PPOTrainer(
    model_bundle=mb,
    prompt_loader=prompt_loader,
    config=config,
)

print("✅ Trainer initialisé")
print(f"   Reward model: {config['reward_model']['name']}")
print(f"   Save dir: {config['logging']['save_dir']}")

# 🔍 Diagnostic: Vérifier que tout est sur le même device
print(f"\n🔍 Diagnostic final - Tous les composants:")
print(f"   Trainer device: {trainer.device}")
print(f"   Policy model: {next(trainer.policy_model.parameters()).device}")
print(f"   Value head: {next(trainer.policy_model.value_head.parameters()).device}")
print(f"   Ref model: {next(trainer.ref_model.parameters()).device}")
print(f"   Reward model: {trainer.reward_model.device}")

⚙️ Initialisation du PPO Trainer...
✓ Gradient checkpointing activé (économie ~30-40% d'activations)
✓ Optimiseur 8-bit activé (économie ~50% sur états optimiseur)
✅ Trainer initialisé
   Reward model: lvwerra/distilbert-imdb
   Save dir: checkpoints/sentiment_ppo

🔍 Diagnostic final - Tous les composants:
   Trainer device: cuda
   Policy model: cuda:0
   Value head: cuda:0
   Ref model: cuda:0
   Reward model: cuda


## 8. Entraînement PPO 

In [14]:
import time

print("="*60)
print(f"🚀 DÉMARRAGE DE L'ENTRAÎNEMENT PPO")
print(f"   Expérience: {config['experiment_name']}")
print(f"   Device: {device}")
print(f"   Epochs: {config['training']['num_epochs']}")
print(f"   Batch size: {config['training']['batch_size']}")
print(f"   Total batches: {len(prompt_loader)}")
print("="*60)

start_time = time.time()

try:
    # Lancer l'entraînement
    trainer.train()
    
    elapsed = time.time() - start_time
    print("\n" + "="*60)
    print(f"✅ Entraînement terminé!")
    print(f"   Temps total: {elapsed/60:.2f} minutes")
    print(f"   Temps moyen par epoch: {elapsed/config['training']['num_epochs']/60:.2f} minutes")
    print("="*60)
    
    # Lien W&B si dispo
    try:
        import wandb
        if getattr(trainer, "use_wandb", False) and hasattr(wandb, "run") and wandb.run:
            print(f"🌐 W&B run: {wandb.run.get_url()}")
    except Exception:
        pass
    
except KeyboardInterrupt:
    print("\n⚠️  Entraînement interrompu par l'utilisateur")
    print("   Les checkpoints partiels ont été sauvegardés")
    
except Exception as e:
    print("\n" + "="*60)
    print(f"❌ ERREUR: {type(e).__name__}")
    print(f"   Message: {e}")
    print("="*60)
    import traceback
    traceback.print_exc()

🚀 DÉMARRAGE DE L'ENTRAÎNEMENT PPO
   Expérience: ppo_sentiment_tinyllama
   Device: cuda
   Epochs: 1
   Batch size: 2
   Total batches: 1249


PPO Epoch 1/1:   0%|          | 0/1249 [00:00<?, ?it/s]

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.12/dist-packages/torch/utils/checkpoint.py:85: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right

: 

: 

## 9. Vérification des Checkpoints

In [ ]:
import glob

save_dir = config['logging']['save_dir']
checkpoints = glob.glob(os.path.join(save_dir, "*.pt"))

if checkpoints:
    print(f"✅ {len(checkpoints)} checkpoint(s) sauvegardé(s):")
    for ckpt in sorted(checkpoints):
        size_mb = os.path.getsize(ckpt) / (1024**2)
        print(f"   📁 {os.path.basename(ckpt)} ({size_mb:.1f} MB)")
else:
    print("⚠️  Aucun checkpoint trouvé")

## 10. Test de Génération Rapide

In [ ]:
# Tester la génération avec le modèle entraîné
test_prompts = [
    "The movie was",
    "I think this product is",
    "The customer service was",
]

print("🧪 Test de génération avec le modèle PPO:")
print("="*60)

mb.policy_model.eval()
with torch.no_grad():
    for prompt in test_prompts:
        inputs = tokenizer(prompt, return_tensors="pt").to(device)
        outputs = mb.policy_model.generate(
            **inputs,
            max_new_tokens=32,
            temperature=0.7,
            do_sample=True,
            pad_token_id=tokenizer.pad_token_id,
        )
        response = tokenizer.decode(outputs[0], skip_special_tokens=True)
        print(f"\n💬 Prompt: {prompt}")
        print(f"✨ Response: {response}")

print("\n" + "="*60)

## 🎯 Prochaines Étapes

Maintenant que l'entraînement est terminé, vous pouvez:

1. **Évaluer le modèle** avec `eval_sentiment2.py`:
   ```bash
   python scripts/eval_sentiment2.py --method ppo --num_samples 3
   ```

2. **Comparer avec DPO et GRPO**:
   ```bash
   python scripts/compare_methods.py
   ```

3. **Visualiser les résultats**:
   ```bash
   python scripts/visualize_comparison.py
   ```

4. **Modifier les hyperparamètres** et réentraîner pour comparer les performances

## 📊 Notes

- Les checkpoints sont sauvegardés dans `checkpoints/sentiment_ppo/`
- Le modèle PPO inclut un **value head** en plus de la policy
- L'entraînement utilise un **reward model** basé sur le sentiment
- Le **target KL** permet l'early stopping pour la stabilité
- Sur CPU, l'entraînement est plus lent mais plus stable que sur MPS (macOS)